In [25]:
import numpy as np
import pandas as pd
import math
import sklearn
from sklearn.preprocessing import MinMaxScaler
import datetime
import os
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

# split data in 80%/10%/10% train/validation/test sets
valid_set_size_percentage = 10 
test_set_size_percentage = 10 

# #display parent directory and working directory
# print(os.path.dirname(os.getcwd())+':', os.listdir(os.path.dirname(os.getcwd())));
# print(os.getcwd()+':', os.listdir(os.getcwd()));

In [26]:
# import all stock prices 
df = pd.read_csv("../input/XPEV.csv", index_col = 0)
df.info()
df

# # number of different stocks
# print('\nnumber of different stocks: ', len(list(set(df.symbol))))
# print(list(set(df.symbol))[:10])

<class 'pandas.core.frame.DataFrame'>
Index: 230 entries, 2020-08-27 to 2021-07-27
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       230 non-null    float64
 1   High       230 non-null    float64
 2   Low        230 non-null    float64
 3   Close      230 non-null    float64
 4   Adj Close  230 non-null    float64
 5   Volume     230 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 12.6+ KB


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-08-27,23.100000,25.000000,21.200001,21.219999,21.219999,82219700
2020-08-28,23.980000,24.400000,22.020000,22.790001,22.790001,44847300
2020-08-31,22.690001,22.790001,20.330000,20.500000,20.500000,20816000
2020-09-01,20.980000,21.790001,20.100000,21.610001,21.610001,15291400
2020-09-02,21.990000,22.000000,20.379999,21.090000,21.090000,9090100
...,...,...,...,...,...,...
2021-07-21,41.619999,44.419998,41.365002,43.869999,43.869999,18107800
2021-07-22,43.549999,43.950001,42.660000,43.270000,43.270000,6905400
2021-07-23,41.990002,42.078999,39.720001,41.000000,41.000000,11451900


In [27]:
df.describe()
# df.columns=["open", "high","low", "close", "adjclose", "volume"]


,Open,High,Low,Close,Adj Close,Volume
count,230.000000,230.000000,230.000000,230.000000,230.000000,2.300000e+02
mean,35.863018,37.244596,34.339417,35.800493,35.800493,2.085706e+07
std,11.286626,11.977375,10.583758,11.334236,11.334236,1.939534e+07
min,17.809999,18.000000,17.110001,17.450001,17.450001,2.789400e+06
25%,27.017499,27.952501,26.062499,27.337500,27.337500,1.063735e+07
50%,36.165001,37.430000,34.600001,35.940000,35.940000,1.557225e+07
75%,43.745001,45.272501,42.162500,44.087498,44.087498,2.352880e+07
max,72.169998,74.489998,63.009998,72.169998,72.169998,1.661462e+08


In [28]:
# choose a specific stock
# drop feature: volume
# normalize stock data
# create train, validation and test data sets

# function to create train, validation, test data given stock data and sequence length
def load_data(stock, seq_len):
    data_raw = stock
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - seq_len): 
        data.append(data_raw[index: index + seq_len])

    data = np.array(data)

    valid_set_size = int(np.round(valid_set_size_percentage/100*data.shape[0]));  
    test_set_size = int(np.round(test_set_size_percentage/100*data.shape[0]));
    train_set_size = data.shape[0] - (valid_set_size + test_set_size);
    
    x_train = data[:train_set_size,:-1,:4]
    y_train = data[:train_set_size,-1,:4]
    
    x_valid = data[train_set_size:train_set_size+valid_set_size,:-1,:4]
    y_valid = data[train_set_size:train_set_size+valid_set_size,-1,:4]
    
    x_test = data[train_set_size+valid_set_size:,:-1,:4]
    y_test = data[train_set_size+valid_set_size:,-1,:4]
    
    return [x_train, y_train, x_valid, y_valid, x_test, y_test]

# choose one stock
df_stock = df.copy()
df_stock = df_stock[['Open', 'High', 'Low', 'Close']]

cols = list(df_stock.columns.values)
print('df_stock.columns.values = ', cols)

# min-max normalization of stock
scaler = MinMaxScaler()
df_stock_norm = df_stock.copy()
df_stock_norm = df_stock_norm.values
df_stock_norm = scaler.fit_transform(df_stock_norm)

# create train, test data
seq_len = 60 # choose sequence length
x_train, y_train, x_valid, y_valid, x_test, y_test = load_data(df_stock_norm, seq_len)
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ', y_train.shape)
print('x_valid.shape = ',x_valid.shape)
print('y_valid.shape = ', y_valid.shape)
print('x_test.shape = ', x_test.shape)
print('y_test.shape = ',y_test.shape)


df_stock.columns.values =  ['Open', 'High', 'Low', 'Close']
x_train.shape =  (136, 59, 4)
y_train.shape =  (136, 4)
x_valid.shape =  (17, 59, 4)
y_valid.shape =  (17, 4)
x_test.shape =  (17, 59, 4)
y_test.shape =  (17, 4)


In [29]:
# 4. Model and validate data 
# RNNs with basic

## Basic Cell RNN in tensorflow

index_in_epoch = 0;
perm_array  = np.arange(x_train.shape[0])
np.random.shuffle(perm_array)

# function to get the next batch
def get_next_batch(batch_size):
    global index_in_epoch, x_train, perm_array   
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > x_train.shape[0]:
        np.random.shuffle(perm_array) # shuffle permutation array
        start = 0 # start next epoch
        index_in_epoch = batch_size
        
    end = index_in_epoch
    return x_train[perm_array[start:end]], y_train[perm_array[start:end]]

# parameters
n_steps = seq_len-1 
n_inputs = 4 
n_neurons = 200 
n_outputs = 4
n_layers = 2
learning_rate = 0.001
batch_size = 50
n_epochs = 100 
train_set_size = x_train.shape[0]
test_set_size = x_test.shape[0]
tf.compat.v1.reset_default_graph()
# tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_outputs])

# use Basic RNN Cell
layers = [tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons, activation=tf.nn.elu)
          for layer in range(n_layers)]

# use Basic LSTM Cell 
#layers = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons, activation=tf.nn.elu)
#          for layer in range(n_layers)]

# use LSTM Cell with peephole connections
#layers = [tf.contrib.rnn.LSTMCell(num_units=n_neurons, 
#                                  activation=tf.nn.leaky_relu, use_peepholes = True)
#          for layer in range(n_layers)]

# use GRU cell
#layers = [tf.contrib.rnn.GRUCell(num_units=n_neurons, activation=tf.nn.leaky_relu)
#          for layer in range(n_layers)]
                                                                     
multi_layer_cell = tf.nn.rnn_cell.MultiRNNCell(layers)
rnn_outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X, dtype=tf.float32)

stacked_rnn_outputs = tf.reshape(rnn_outputs, [-1, n_neurons]) 
stacked_outputs = tf.layers.dense(stacked_rnn_outputs, n_outputs)
outputs = tf.reshape(stacked_outputs, [-1, n_steps, n_outputs])
outputs = outputs[:,n_steps-1,:] # keep only last output of sequence
                                              
loss = tf.reduce_mean(tf.square(outputs - y)) # loss function = mean squared error 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate) 
training_op = optimizer.minimize(loss)
                                              
# run graph
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer())
    for iteration in range(int(n_epochs*train_set_size/batch_size)):
        x_batch, y_batch = get_next_batch(batch_size) # fetch the next training batch 
        
        # sess.run(training_op, feed_dict={X: x_batch.reshape(((50 , 19, 4))), y: y_batch.reshape(((, 4)))}) 
        sess.run(training_op, feed_dict={X: x_batch, y: y_batch}) 

        if iteration % int(5*train_set_size/batch_size) == 0:
            mse_train = loss.eval(feed_dict={X: x_train, y: y_train}) 
            mse_valid = loss.eval(feed_dict={X: x_valid, y: y_valid}) 
            print('%.2f epochs: MSE train/valid = %.6f/%.6f'%(
                iteration*batch_size/train_set_size, mse_train, mse_valid))

    y_train_pred = sess.run(outputs, feed_dict={X: x_train})
    y_valid_pred = sess.run(outputs, feed_dict={X: x_valid})
    y_test_pred = sess.run(outputs, feed_dict={X: x_test})
    


/Users/anhnguyenhoang/miniconda/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:427: UserWarning: `tf.nn.rnn_cell.BasicRNNCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.SimpleRNNCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicRNNCell` is deprecated and will be "
/Users/anhnguyenhoang/miniconda/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
/Users/anhnguyenhoang/miniconda/lib/python3.8/site-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is depre

0.00 epochs: MSE train/valid = 0.266518/0.242574
4.78 epochs: MSE train/valid = 0.010087/0.004702
9.56 epochs: MSE train/valid = 0.006028/0.001964
14.34 epochs: MSE train/valid = 0.003844/0.001722
19.12 epochs: MSE train/valid = 0.002692/0.001536
23.90 epochs: MSE train/valid = 0.002269/0.001546
28.68 epochs: MSE train/valid = 0.001834/0.001626
33.46 epochs: MSE train/valid = 0.001572/0.001521
38.24 epochs: MSE train/valid = 0.001441/0.001639
43.01 epochs: MSE train/valid = 0.001424/0.002266
47.79 epochs: MSE train/valid = 0.001086/0.001561
52.57 epochs: MSE train/valid = 0.000930/0.001592
57.35 epochs: MSE train/valid = 0.000908/0.002003
62.13 epochs: MSE train/valid = 0.000935/0.001713
66.91 epochs: MSE train/valid = 0.000787/0.001676
71.69 epochs: MSE train/valid = 0.000845/0.001952
76.47 epochs: MSE train/valid = 0.001009/0.002773
81.25 epochs: MSE train/valid = 0.000821/0.001689
86.03 epochs: MSE train/valid = 0.001079/0.002352
90.81 epochs: MSE train/valid = 0.000709/0.002133
95.

In [42]:
y_test_actual = scaler.inverse_transform(y_test)
y_test_predictions = scaler.inverse_transform(y_test_pred)
save = pd.DataFrame({'Close':y_test_actual[:,3],'Prediction':y_test_predictions[:,3]})
save.index= df.index[-y_test.shape[0]-1:-1]
save.to_csv('../out/RNN/XPEV.csv')